In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
import json

# Load the VGG19 model
vgg19 = VGG19(weights='C:/Users/K-Gen/Desktop/Reacting/vgg19/versions/2/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5', include_top=False)

# Define content and style layers
content_layers = ['block5_conv2']
style_layers = ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1', 'block5_conv1']
num_content_layers = len(content_layers)
num_style_layers = len(style_layers)

# Create the model
def get_model():
    vgg = VGG19(weights='C:/Users/K-Gen/Desktop/Reacting/vgg19/versions/2/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5', include_top=False)
    vgg.trainable = False
    outputs = [vgg.get_layer(name).output for name in style_layers + content_layers]
    model = tf.keras.Model([vgg.input], outputs)
    return model

# Load class index
with open('C:/Users/K-Gen/Desktop/Reacting/vgg19/versions/2/imagenet_class_index.json') as f:
    class_idx = json.load(f)
idx2label = [class_idx[str(k)][1] for k in range(len(class_idx))]

def gram_matrix(input_tensor):
    result = tf.linalg.einsum('bijc,bijd->bcd', input_tensor, input_tensor)
    input_shape = tf.shape(input_tensor)
    num_locations = tf.cast(input_shape[1]*input_shape[2], tf.float32)
    return result/(num_locations)

class StyleContentModel(tf.keras.models.Model):
    def __init__(self, style_layers, content_layers):
        super(StyleContentModel, self).__init__()
        self.vgg = get_model()
        self.style_layers = style_layers
        self.content_layers = content_layers
        self.num_style_layers = len(style_layers)
        self.vgg.trainable = False

    def call(self, inputs):
        inputs = inputs*255.0
        preprocessed_input = preprocess_input(inputs)
        outputs = self.vgg(preprocessed_input)
        style_outputs, content_outputs = (outputs[:self.num_style_layers],
                                          outputs[self.num_style_layers:])

        style_outputs = [gram_matrix(style_output)
                         for style_output in style_outputs]

        content_dict = {content_name: value
                        for content_name, value
                        in zip(self.content_layers, content_outputs)}

        style_dict = {style_name: value
                      for style_name, value
                      in zip(self.style_layers, style_outputs)}

        return {'content': content_dict, 'style': style_dict}

# Initialize the model
extractor = StyleContentModel(style_layers, content_layers)

# Open the camera
cap = cv2.VideoCapture(0)

# Load and preprocess the style image (steampunk style)
style_image = cv2.imread('st.jpg')
style_image = cv2.cvtColor(style_image, cv2.COLOR_BGR2RGB)
style_image = style_image.astype(np.float32) / 255.0
style_image = np.expand_dims(style_image, axis=0)

def style_transfer(content_image, style_image, num_iterations=10):
    content_image = tf.convert_to_tensor(content_image, dtype=tf.float32)
    style_image = tf.convert_to_tensor(style_image, dtype=tf.float32)
    
    style_targets = extractor(style_image)['style']
    content_targets = extractor(content_image)['content']
    
    image = tf.Variable(content_image)
    
    opt = tf.optimizers.Adam(learning_rate=0.02, beta_1=0.99, epsilon=1e-1)
    style_weight = 1e-2
    content_weight = 1e4

    @tf.function()
    def train_step(image):
        with tf.GradientTape() as tape:
            outputs = extractor(image)
            loss = style_content_loss(outputs, style_targets, content_targets, style_weight, content_weight)

        grad = tape.gradient(loss, image)
        opt.apply_gradients([(grad, image)])
        image.assign(tf.clip_by_value(image, 0.0, 1.0))

    for _ in range(num_iterations):
        train_step(image)
    
    return image.numpy()

def style_content_loss(outputs, style_targets, content_targets, style_weight, content_weight):
    style_outputs = outputs['style']
    content_outputs = outputs['content']
    style_loss = tf.add_n([tf.reduce_mean((style_outputs[name]-style_targets[name])**2) 
                           for name in style_outputs.keys()])
    style_loss *= style_weight / num_style_layers

    content_loss = tf.add_n([tf.reduce_mean((content_outputs[name]-content_targets[name])**2) 
                             for name in content_outputs.keys()])
    content_loss *= content_weight / num_content_layers
    loss = style_loss + content_loss
    return loss

def apply_steampunk_effects(image):
    # Apply sepia tone
    sepia_matrix = np.array([[0.393, 0.769, 0.189],
                             [0.349, 0.686, 0.168],
                             [0.272, 0.534, 0.131]])
    sepia_image = cv2.transform(image, sepia_matrix)
    
    # Add vignette effect
    rows, cols = image.shape[:2]
    kernel_x = cv2.getGaussianKernel(cols, 200)
    kernel_y = cv2.getGaussianKernel(rows, 200)
    kernel = kernel_y * kernel_x.T
    mask = 255 * kernel / np.linalg.norm(kernel)
    vignette = np.copy(sepia_image)
    for i in range(3):
        vignette[:, :, i] = vignette[:, :, i] * mask
    
    # Blend original and vignette
    result = cv2.addWeighted(sepia_image, 0.7, vignette, 0.3, 0)
    
    return result

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Preprocess the frame
    input_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) / 255.0
    input_frame = np.expand_dims(input_frame, axis=0)
    
    # Apply style transfer
    stylized_frame = style_transfer(input_frame, style_image, num_iterations=1)
    
    # Post-process the stylized frame
    output_frame = np.squeeze(stylized_frame)
    output_frame = (output_frame * 255).astype(np.uint8)
    output_frame = cv2.cvtColor(output_frame, cv2.COLOR_RGB2BGR)
    
    # Apply additional steampunk effects
    final_frame = apply_steampunk_effects(output_frame)
    
    # Display the result
    cv2.imshow('Steampunk Camera', final_frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [ ]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print("Devices:", tf.config.list_physical_devices())